# CI/CD & Team Working - Comprehensive Notes

## CI/CD Overview

### What is CI/CD?

**Continuous Integration (CI)** is the practice of automatically building and testing code changes as they're committed to version control.

**Continuous Deployment (CD)** is the practice of automatically deploying code that passes all tests to production.

### Key Benefits
- **Early bug detection**: Automated tests catch issues before they reach production
- **Faster release cycles**: Automation reduces manual deployment time
- **Improved code quality**: Consistent testing and quality checks
- **Reduced risk**: Small, frequent changes are easier to troubleshoot
- **Team collaboration**: Shared standards and automated workflows

### CI/CD Pipeline Stages

1. **Source**: Code committed to version control (Git)
2. **Build**: Code compiled and dependencies installed
3. **Test**: Automated tests run (unit, integration, end-to-end)
4. **Deploy**: Code deployed to staging/production environments
5. **Monitor**: Track application performance and errors

---



## GitHub Actions

### Overview
GitHub Actions is a CI/CD platform integrated directly into GitHub repositories. It automates workflows based on repository events.

### Key Concepts

#### Workflows
- YAML files stored in `.github/workflows/`
- Define automated processes triggered by events
- Can run on different operating systems (Ubuntu, Windows, macOS)

#### Events (Triggers)
- `push`: Code pushed to repository
- `pull_request`: PR opened, synchronized, or reopened
- `schedule`: Cron-based scheduling
- `workflow_dispatch`: Manual trigger
- `release`: New release created

#### Jobs
- Independent tasks that run in parallel (by default)
- Each job runs on a fresh virtual machine (runner)
- Can be configured to run sequentially using `needs`

#### Steps
- Individual tasks within a job
- Can run commands or use actions
- Execute sequentially within a job

#### Actions
- Reusable units of code
- Can be from GitHub Marketplace or custom
- Common examples: `actions/checkout@v3`, `actions/setup-python@v4`

### Basic Workflow Example

```yaml
name: CI Pipeline

on:
  push:
    branches: [ main, develop ]
  pull_request:
    branches: [ main ]

jobs:
  test:
    runs-on: ubuntu-latest
    
    steps:
    - name: Checkout code
      uses: actions/checkout@v3
    
    - name: Set up Python
      uses: actions/setup-python@v4
      with:
        python-version: '3.11'
    
    - name: Install dependencies
      run: |
        python -m pip install --upgrade pip
        pip install -r requirements.txt
    
    - name: Run tests
      run: |
        pytest --cov=app --cov-report=term-missing
    
    - name: Upload coverage
      uses: codecov/codecov-action@v3
      if: success()
```

### Advanced Workflow Features

#### Matrix Strategy
Run tests across multiple versions/configurations:

```yaml
jobs:
  test:
    runs-on: ubuntu-latest
    strategy:
      matrix:
        python-version: ['3.9', '3.10', '3.11']
        
    steps:
    - uses: actions/setup-python@v4
      with:
        python-version: ${{ matrix.python-version }}
```

#### Environment Variables and Secrets

```yaml
jobs:
  deploy:
    runs-on: ubuntu-latest
    env:
      DATABASE_URL: ${{ secrets.DATABASE_URL }}
      API_KEY: ${{ secrets.OPENWEATHERMAP_API_KEY }}
    
    steps:
    - name: Deploy application
      run: |
        echo "Deploying with API key"
```

#### Conditional Execution

```yaml
steps:
- name: Run only on main branch
  if: github.ref == 'refs/heads/main'
  run: echo "Main branch deployment"

- name: Run on PR
  if: github.event_name == 'pull_request'
  run: echo "PR validation"
```

#### Caching Dependencies

```yaml
- name: Cache Python dependencies
  uses: actions/cache@v3
  with:
    path: ~/.cache/pip
    key: ${{ runner.os }}-pip-${{ hashFiles('**/requirements.txt') }}
    restore-keys: |
      ${{ runner.os }}-pip-
```

---



## Deployment Strategies

### 1. Blue-Green Deployment
- Maintain two identical production environments (Blue and Green)
- Deploy new version to inactive environment
- Switch traffic after validation
- **Advantage**: Instant rollback, zero downtime
- **Disadvantage**: Requires double infrastructure

### 2. Rolling Deployment
- Gradually replace instances with new version
- Update servers one-by-one or in batches
- **Advantage**: No extra infrastructure needed
- **Disadvantage**: Mixed versions during deployment

### 3. Canary Deployment
- Deploy to small subset of users first
- Monitor metrics before full rollout
- **Advantage**: Risk mitigation, early feedback
- **Disadvantage**: Complex routing and monitoring

### 4. Feature Flags
- Deploy code with features disabled
- Enable features for specific users/groups
- **Advantage**: Decouple deployment from release
- **Disadvantage**: Code complexity increases

### Deployment Checklist

**Pre-Deployment**
- [ ] All tests pass
- [ ] Code review completed
- [ ] Database migrations tested
- [ ] Configuration updated
- [ ] Rollback plan documented

**During Deployment**
- [ ] Monitor error rates
- [ ] Check application logs
- [ ] Verify health endpoints
- [ ] Test critical user flows

**Post-Deployment**
- [ ] Monitor performance metrics
- [ ] Check error tracking (Sentry, etc.)
- [ ] Verify database connections
- [ ] Announce deployment to team

### Environment Management

```yaml
# Example: Multiple environments
environments:
  development:
    database: dev-db
    api_key: dev-key
    
  staging:
    database: staging-db
    api_key: staging-key
    
  production:
    database: prod-db
    api_key: prod-key
```

---

## Testing in CI/CD Pipeline

### Testing Pyramid

```
        /\
       /E2E\      Few, slow, expensive
      /------\
     /Integration\   More, moderate speed
    /--------------\
   /   Unit Tests   \ Many, fast, cheap
  /------------------\
```

### Unit Tests
- Test individual functions/methods in isolation
- Use mocking to isolate dependencies
- Fast execution, high coverage

**Example from project:**
```python
# test_utils.py
def test_add_two_positive(self):
    # Arrange
    # Action
    result = add(1, 2)
    expected = 3
    # Assert
    assert result == expected
```

### Integration Tests
- Test interaction between components
- Mock external dependencies (APIs, databases)
- Verify data flow between layers

**Example from project:**
```python
# test_services.py
def test_get_weather(self, service, mock_db, mock_api_client):
    # Arrange
    mock_api_client.get_current_weather.return_value = {...}
    
    # Action
    result = service.get_weather('Tokyo')
    
    # Assert
    assert result['city'] == 'Tokyo'
    mock_api_client.get_current_weather.assert_called_once_with('Tokyo')
    mock_db.add.assert_called_once()
```

### End-to-End Tests
- Test complete user flows
- Use real or staging environments
- Verify system behavior from user perspective

**Example from project:**
```python
# test_endpoints.py
def test_get_weather_point(self, client, mock_service):
    # Arrange
    mock_service.get_weather.return_value = {...}
    app.dependency_overrides[get_weather_service] = lambda: mock_service
    
    # Action
    response = client.get('/weather/Tokyo')
    
    # Assert
    assert response.status_code == 200
    assert response.json()['city'] == 'Tokyo'
```

### Mocking Best Practices

**When to Mock:**
- External API calls (WeatherAPIClient)
- Database connections
- File system operations
- Time-dependent functions
- Expensive computations

**Mocking Strategies:**

```python
# 1. Mock at function level
@patch('app.external_api.requests.get')
def test_api_call(mock_get):
    mock_get.return_value = MagicMock()

# 2. Mock at fixture level
@pytest.fixture
def mock_db(self):
    return MagicMock(spec=Session)

# 3. Override dependencies (FastAPI)
app.dependency_overrides[get_weather_service] = lambda: mock_service
```

### Test Coverage

```bash
# Run tests with coverage report
pytest --cov=app --cov-report=term-missing

# Generate HTML coverage report
pytest --cov=app --cov-report=html

# Coverage goals
# - Unit tests: 80-90%
# - Critical paths: 100%
# - Overall project: 70%+
```

### Test Organization

```
tests/
├── __init__.py
├── test_utils.py           # Unit tests for utilities
├── test_services.py        # Service layer tests
├── test_external_api.py    # External API tests
├── test_endpoints.py       # API endpoint tests
└── conftest.py             # Shared fixtures
```

---

## Agile & Scrum

### Agile Principles

Agile is a methodology focused on iterative development, collaboration, and flexibility.

**Core Values:**
1. **Individuals and interactions** over processes and tools
2. **Working software** over comprehensive documentation
3. **Customer collaboration** over contract negotiation
4. **Responding to change** over following a plan

### Scrum Framework

Scrum is an agile framework that organizes work into time-boxed iterations called sprints.

#### Scrum Roles

**Product Owner**
- Defines product vision and roadmap
- Manages product backlog
- Prioritizes features based on business value
- Accepts or rejects work results

**Scrum Master**
- Facilitates scrum ceremonies
- Removes impediments/blockers
- Coaches team on agile practices
- Shields team from interruptions

**Development Team**
- Cross-functional (developers, testers, designers)
- Self-organizing and self-managing
- 3-9 members (ideal size)
- Collectively responsible for deliverables

#### Scrum Ceremonies

**Sprint Planning** (2-4 hours for 2-week sprint)
- Select items from product backlog
- Define sprint goal
- Break down user stories into tasks
- Estimate effort required

**Daily Standup** (15 minutes)
- What did I complete yesterday?
- What will I work on today?
- Are there any blockers?
- *Note: Not a status report, but coordination*

**Sprint Review** (1-2 hours)
- Demo completed work to stakeholders
- Gather feedback
- Discuss what was and wasn't completed
- Update product backlog

**Sprint Retrospective** (1-1.5 hours)
- What went well?
- What could be improved?
- What actions will we take next sprint?
- Focus on continuous improvement

**Backlog Refinement** (Ongoing)
- Clarify user stories
- Add acceptance criteria
- Break down large stories
- Estimate story points

#### Scrum Artifacts

**Product Backlog**
- Prioritized list of features/requirements
- Managed by Product Owner
- Continuously refined and updated
- Items described as user stories

**Sprint Backlog**
- Subset of product backlog for current sprint
- Owned by development team
- Updated daily
- Includes tasks and progress

**Increment**
- Sum of all completed backlog items
- Must be "Done" (meets Definition of Done)
- Potentially shippable product
- Demonstrated in Sprint Review

#### User Stories

Format: "As a [role], I want [feature] so that [benefit]"

```
User Story: Weather History
As a user, I want to view historical weather data 
so that I can see temperature trends over time.

Acceptance Criteria:
- Display weather logs for past 7 days by default
- Allow filtering by number of days (1-30)
- Show temperature, description, and timestamp
- Format dates in user's local timezone
```

#### Definition of Done (DoD)

Checklist for completing work:
- [ ] Code written and reviewed
- [ ] Unit tests written and passing
- [ ] Integration tests passing
- [ ] Documentation updated
- [ ] No critical bugs
- [ ] Deployed to staging
- [ ] Accepted by Product Owner

#### Story Points & Estimation

**Fibonacci Sequence**: 1, 2, 3, 5, 8, 13, 21
- Represents relative complexity, not time
- Smaller numbers = more precise estimates
- Larger numbers = more uncertainty

**Planning Poker**
- Team members independently estimate
- Discuss differences
- Re-estimate until consensus

### Agile Metrics

**Velocity**
- Story points completed per sprint
- Used to predict future capacity
- Improves accuracy over time

**Burndown Chart**
- Visual representation of work remaining
- Shows progress toward sprint goal
- Helps identify if team is on track

**Lead Time**
- Time from story creation to completion
- Measures overall process efficiency

**Cycle Time**
- Time from "In Progress" to "Done"
- Measures development efficiency

---

## Kanban Board

### Overview
Kanban is a visual workflow management method that helps teams visualize work, limit work in progress, and maximize efficiency.

### Kanban Principles

1. **Visualize workflow**: Make all work visible
2. **Limit WIP**: Control work in progress
3. **Manage flow**: Optimize throughput
4. **Make policies explicit**: Clear definition of each stage
5. **Implement feedback loops**: Regular reviews
6. **Improve collaboratively**: Continuous improvement

### Basic Kanban Board Structure

```
┌─────────┬─────────┬─────────┬─────────┬──────┐
│ Backlog │  To Do  │ In Prog │ Review  │ Done │
├─────────┼─────────┼─────────┼─────────┼──────┤
│ Task A  │ Task D  │ Task F  │ Task H  │Task J│
│ Task B  │ Task E  │ Task G  │         │Task K│
│ Task C  │         │         │         │      │
│   ...   │         │         │         │      │
└─────────┴─────────┴─────────┴─────────┴──────┘
WIP Limit:    ∞         3         2         2      ∞
```

### Common Column Structures

**Simple Board:**
- To Do → In Progress → Done

**Standard Board:**
- Backlog → To Do → In Progress → Review → Done

**Development Board:**
- Backlog → Ready → Dev → Code Review → Testing → Staging → Production

**Detailed Board:**
- Backlog → Refined → To Do → In Progress → PR Open → Review → Testing → Deploy → Done

### Work in Progress (WIP) Limits

**Purpose:**
- Prevent multitasking
- Identify bottlenecks
- Improve focus and flow
- Reduce context switching

**Setting WIP Limits:**
- Start with: (Number of team members × 1.5)
- Adjust based on observation
- Different limits for different columns
- Exception: Backlog and Done typically unlimited

**Example for 3-person team:**
```
To Do: 4
In Progress: 3
Review: 2
Testing: 2
```

### Kanban Cards

**Card Information:**
- Title/Description
- Assignee(s)
- Priority (High, Medium, Low)
- Labels/Tags (bug, feature, tech-debt)
- Due date
- Estimate (optional)
- Blocked status

**Example Card:**
```
┌─────────────────────────────────┐
│ 🔴 HIGH                         │
│ Implement Weather History API   │
│                                 │
│ Assignee: @developer            │
│ Labels: feature, backend        │
│ Estimate: 5 points              │
│ Due: Dec 15                     │
└─────────────────────────────────┘
```

### Kanban Metrics

**Throughput**
- Number of items completed per time period
- Measure: Cards moved to Done per week

**Lead Time**
- Total time from card creation to completion
- Goal: Reduce over time

**Cycle Time**
- Time from "In Progress" to "Done"
- Measure: Average days in active development

**Cumulative Flow Diagram (CFD)**
- Visualizes work distribution across columns
- Identifies bottlenecks
- Shows WIP trends

### Kanban vs Scrum

| Aspect | Kanban | Scrum |
|--------|--------|-------|
| **Iterations** | Continuous flow | Fixed sprints (1-4 weeks) |
| **Roles** | No required roles | Product Owner, Scrum Master, Team |
| **Changes** | Anytime | Only between sprints |
| **WIP Limits** | Explicit limits | Implicit (sprint capacity) |
| **Metrics** | Lead time, cycle time | Velocity, burndown |
| **Ceremonies** | Optional | Required (daily standup, etc.) |
| **Best for** | Support/maintenance | Product development |

### Digital Kanban Tools

**GitHub Projects**
- Integrated with repositories
- Automation rules
- Multiple views (board, table, roadmap)

**Jira**
- Advanced reporting
- Custom workflows
- Integration ecosystem

**Trello**
- Simple and visual
- Power-ups for extensions
- Good for small teams

**Linear**
- Fast and keyboard-friendly
- Cycle-based planning
- Developer-focused

### GitHub Projects Example

```yaml
# .github/workflows/project-automation.yml
name: Project Automation

on:
  issues:
    types: [opened]
  pull_request:
    types: [opened, closed]

jobs:
  add-to-project:
    runs-on: ubuntu-latest
    steps:
      - name: Add issue to project
        uses: actions/add-to-project@v0.4.0
        with:
          project-url: https://github.com/orgs/ORG/projects/1
```

### Best Practices

1. **Keep cards small**: 1-3 days of work maximum
2. **Update regularly**: Move cards as work progresses
3. **Review WIP limits**: Adjust based on bottlenecks
4. **Daily review**: Team checks board together
5. **Block explicitly**: Mark blocked items clearly
6. **Archive done work**: Keep board focused
7. **Use labels**: Categorize work types
8. **Document policies**: Clear column definitions

---

## Microservices Architecture

### Overview
Microservices architecture breaks down applications into small, independent services that communicate through well-defined APIs.

### Monolith vs Microservices

**Monolithic Architecture:**
```
┌─────────────────────────────────┐
│     Single Application          │
│  ┌────────────────────────────┐ │
│  │ Weather Service            │ │
│  │ User Management            │ │
│  │ Notification Service       │ │
│  │ Shared Database            │ │
│  └────────────────────────────┘ │
└─────────────────────────────────┘
```

**Microservices Architecture:**
```
┌──────────────┐  ┌──────────────┐  ┌──────────────┐
│   Weather    │  │    User      │  │ Notification │
│   Service    │  │  Management  │  │   Service    │
├──────────────┤  ├──────────────┤  ├──────────────┤
│ Weather DB   │  │  User DB     │  │  Message DB  │
└──────────────┘  └──────────────┘  └──────────────┘
       ↓                 ↓                   ↓
       └─────────── API Gateway ────────────┘
```

### Microservices Characteristics

1. **Independent deployment**: Each service can be deployed separately
2. **Technology diversity**: Different services can use different tech stacks
3. **Organized around business capabilities**: Each service represents a domain
4. **Decentralized data management**: Each service owns its database
5. **Fault isolation**: Failure in one service doesn't crash entire system
6. **Scalability**: Scale individual services based on demand

### Service Communication

#### Synchronous Communication (REST/HTTP)

```python
# Weather Service API
@app.get("/weather/{city}")
def get_weather(city: str):
    return {"city": city, "temperature": 20}

# Client calling Weather Service
import requests
response = requests.get(f"http://weather-service/weather/{city}")
weather = response.json()
```

**Advantages:**
- Simple and straightforward
- Immediate response
- Easy to debug

**Disadvantages:**
- Tight coupling
- Service dependency
- Cascading failures possible

#### Asynchronous Communication (Message Queues)

```python
# Producer (Weather Service)
from celery import Celery
celery = Celery('tasks', broker='redis://localhost:6379')

@celery.task
def process_weather_data(city):
    # Process weather data
    return {"city": city, "status": "processed"}

# Consumer (Notification Service)
@celery.task
def send_weather_alert(weather_data):
    # Send notification based on weather
    pass
```

**Advantages:**
- Loose coupling
- Better fault tolerance
- Natural load balancing

**Disadvantages:**
- Eventual consistency
- More complex debugging
- Message ordering challenges

### API Gateway Pattern

API Gateway acts as single entry point for all clients.

**Responsibilities:**
- Request routing
- Authentication/Authorization
- Rate limiting
- Load balancing
- Response transformation
- Caching

**Example with FastAPI:**
```python
# api_gateway.py
from fastapi import FastAPI, HTTPException
import httpx

app = FastAPI()

WEATHER_SERVICE = "http://weather-service:8000"
USER_SERVICE = "http://user-service:8001"

@app.get("/api/weather/{city}")
async def get_weather(city: str, user_id: str):
    # Verify user
    async with httpx.AsyncClient() as client:
        user_resp = await client.get(f"{USER_SERVICE}/users/{user_id}")
        if user_resp.status_code != 200:
            raise HTTPException(401, "Unauthorized")
        
        # Get weather data
        weather_resp = await client.get(f"{WEATHER_SERVICE}/weather/{city}")
        return weather_resp.json()
```

### Service Discovery

Services need to find each other dynamically.

**Solutions:**
- **Consul**: Service mesh with health checking
- **Eureka**: Netflix's service registry
- **Kubernetes DNS**: Built-in service discovery
- **Environment variables**: Simple for small setups

### Database per Service Pattern

Each microservice owns and manages its own database.

```
Weather Service → PostgreSQL (weather_logs)
User Service → PostgreSQL (users, sessions)
Notification Service → Redis (message queue)
Analytics Service → MongoDB (aggregated data)
```

**Advantages:**
- Service independence
- Technology flexibility
- Scalability

**Challenges:**
- Distributed transactions
- Data consistency
- Querying across services

### Handling Distributed Transactions

#### Saga Pattern

Break transaction into sequence of local transactions.

```python
# Order Saga Example
class OrderSaga:
    def create_order(self, order_data):
        try:
            # Step 1: Reserve inventory
            inventory_id = self.inventory_service.reserve(order_data)
            
            # Step 2: Process payment
            payment_id = self.payment_service.charge(order_data)
            
            # Step 3: Create order
            order_id = self.order_service.create(order_data)
            
            return {"order_id": order_id}
        except Exception as e:
            # Compensating transactions
            self.inventory_service.release(inventory_id)
            self.payment_service.refund(payment_id)
            raise
```

### Circuit Breaker Pattern

Prevent cascading failures by detecting and handling failures.

```python
from circuitbreaker import circuit

@circuit(failure_threshold=5, recovery_timeout=60)
def call_weather_service(city):
    response = requests.get(f"http://weather-service/weather/{city}")
    response.raise_for_status()
    return response.json()

# If 5 failures occur, circuit opens
# Requests fail fast for 60 seconds
# Then attempts recovery
```

### Monitoring and Observability

#### Distributed Tracing
Track requests across multiple services.

```python
# Using OpenTelemetry
from opentelemetry import trace

tracer = trace.get_tracer(__name__)

@app.get("/weather/{city}")
def get_weather(city: str):
    with tracer.start_as_current_span("get_weather"):
        with tracer.start_as_current_span("call_external_api"):
            result = api_client.get_current_weather(city)
        
        with tracer.start_as_current_span("save_to_db"):
            db.add(WeatherLog(**result))
            db.commit()
        
        return result
```

#### Centralized Logging
Aggregate logs from all services.

```python
import logging
import json

logger = logging.getLogger(__name__)

def log_request(request_id, service_name, action):
    logger.info(json.dumps({
        "request_id": request_id,
        "service": service_name,
        "action": action,
        "timestamp": datetime.now().isoformat()
    }))
```

**Tools:**
- **ELK Stack**: Elasticsearch, Logstash, Kibana
- **Grafana Loki**: Log aggregation
- **CloudWatch**: AWS logging solution

#### Health Checks

```python
@app.get("/health")
def health_check():
    return {
        "status": "healthy",
        "database": check_database_connection(),
        "cache": check_redis_connection(),
        "external_api": check_api_availability()
    }
```

### Container Orchestration

#### Docker Compose (Development)

```yaml
# docker-compose.yml
version: '3.8'

services:
  weather-service:
    build: ./weather-service
    ports:
      - "8000:8000"
    environment:
      - DATABASE_URL=postgresql://db:5432/weather
      - REDIS_URL=redis://redis:6379
    depends_on:
      - db
      - redis
  
  user-service:
    build: ./user-service
    ports:
      - "8001:8000"
    environment:
      - DATABASE_URL=postgresql://db:5432/users
  
  db:
    image: postgres:15
    environment:
      - POSTGRES_PASSWORD=secret
  
  redis:
    image: redis:7
```

#### Kubernetes (Production)

```yaml
# weather-service-deployment.yaml
apiVersion: apps/v1
kind: Deployment
metadata:
  name: weather-service
spec:
  replicas: 3
  selector:
    matchLabels:
      app: weather-service
  template:
    metadata:
      labels:
        app: weather-service
    spec:
      containers:
      - name: weather-service
        image: weather-service:latest
        ports:
        - containerPort: 8000
        env:
        - name: DATABASE_URL
          valueFrom:
            secretKeyRef:
              name: weather-secrets
              key: database-url
---
apiVersion: v1
kind: Service
metadata:
  name: weather-service
spec:
  selector:
    app: weather-service
  ports:
  - port: 80
    targetPort: 8000
  type: LoadBalancer
```

### Microservices Best Practices

1. **Start with monolith**: Don't begin with microservices
2. **Define clear boundaries**: Based on business domains
3. **API versioning**: Support backward compatibility
4. **Automated testing**: Essential for distributed systems
5. **Monitoring**: Observability from day one
6. **Documentation**: Clear API contracts
7. **Security**: Authentication at gateway, authorization per service
8. **Data consistency**: Plan for eventual consistency
9. **Deployment automation**: CI/CD is mandatory
10. **Team structure**: Align teams with services (Conway's Law)

### When to Use Microservices

**Good fit:**
- Large, complex applications
- Multiple teams working independently
- Different scalability requirements
- Need for technology diversity
- Frequent deployments

**Not a good fit:**
- Small applications
- Simple CRUD operations
- Single small team
- Limited DevOps capability
- Tight latency requirements

### Example: Weather Application as Microservices

```
┌─────────────────────┐
│    API Gateway      │
│   (FastAPI/Nginx)   │
└──────────┬──────────┘
           │
    ┌──────┴──────┬──────────┬────────────┐
    ↓             ↓          ↓            ↓
┌─────────┐  ┌────────┐  ┌────────┐  ┌────────┐
│ Weather │  │  User  │  │History │  │ Alert  │
│ Service │  │Service │  │Service │  │Service │
└────┬────┘  └───┬────┘  └───┬────┘  └───┬────┘
     │           │            │            │
┌────▼────┐ ┌───▼────┐  ┌───▼────┐  ┌───▼────┐
│Weather  │ │ User   │  │History │  │Message │
│   DB    │ │   DB   │  │   DB   │  │ Queue  │
└─────────┘ └────────┘  └────────┘  └────────┘
```

**Service Responsibilities:**

- **Weather Service**: Fetch current weather from external API
- **User Service**: Authentication, authorization, user profiles
- **History Service**: Store and retrieve historical weather data
- **Alert Service**: Send notifications based on weather conditions

---



## Integration: CI/CD + Agile + Microservices

### Complete Development Workflow

1. **Planning (Agile)**
   - Product Owner prioritizes user stories
   - Team estimates during sprint planning
   - Stories added to Kanban board

2. **Development**
   - Developer picks story from "To Do"
   - Creates feature branch
   - Develops microservice changes
   - Writes tests (TDD)
   - Moves card to "In Progress"

3. **Code Review**
   - Opens pull request
   - CI pipeline runs automatically
   - GitHub Actions executes tests
   - Code review by peers
   - Moves card to "Review"

4. **Testing**
   - Automated tests in CI
   - Manual QA if needed
   - Integration tests across services
   - Moves card to "Testing"

5. **Deployment (CD)**
   - Merge to main branch
   - Automated deployment pipeline
   - Deploy to staging
   - Run smoke tests
   - Deploy to production
   - Moves card to "Done"

6. **Monitoring**
   - Track metrics in production
   - Monitor service health
   - Gather user feedback
   - Plan next iteration

### GitHub Actions for Microservices

```yaml
name: Microservices CI/CD

on:
  push:
    branches: [main]
    paths:
      - 'services/weather/**'

jobs:
  test:
    runs-on: ubuntu-latest
    steps:
    - uses: actions/checkout@v3
    
    - name: Run tests
      run: |
        cd services/weather
        pytest --cov
    
  build:
    needs: test
    runs-on: ubuntu-latest
    steps:
    - name: Build Docker image
      run: |
        docker build -t weather-service:${{ github.sha }} .
    
    - name: Push to registry
      run: |
        docker push weather-service:${{ github.sha }}
  
  deploy:
    needs: build
    runs-on: ubuntu-latest
    steps:
    - name: Deploy to Kubernetes
      run: |
        kubectl set image deployment/weather-service \
          weather-service=weather-service:${{ github.sha }}
```

---



## Summary Checklist

### CI/CD Essentials
- Automated testing in pipeline
- Code coverage tracking
- Automated deployment to staging
- Manual approval for production
- Rollback strategy defined
- Secrets management configured

### Agile/Scrum Essentials
- Sprint length defined (1-2 weeks typical)
- Daily standup scheduled
- Sprint ceremonies on calendar
- User stories with acceptance criteria
- Definition of Done documented
- Retrospective action items tracked

### Kanban Essentials
- Board columns defined
- WIP limits set
- Card template created
- Automation rules configured
- Metrics dashboard set up
- Regular board reviews scheduled

### Microservices Essentials
- Service boundaries defined
- API contracts documented
- Inter-service communication pattern chosen
- Health check endpoints implemented
- Monitoring and logging configured
- CI/CD pipeline per service
- Database per service pattern
- Error handling and circuit breakers

---



## Additional Resources

### Documentation
- GitHub Actions: https://docs.github.com/actions
- Pytest: https://docs.pytest.org
- FastAPI: https://fastapi.tiangolo.com
- Scrum Guide: https://scrumguides.org
- Microservices.io: https://microservices.io

### Tools
- **CI/CD**: GitHub Actions, GitLab CI, Jenkins, CircleCI
- **Testing**: pytest, unittest, Jest, Selenium
- **Kanban**: GitHub Projects, Jira, Trello, Linear
- **Containers**: Docker, Kubernetes, Docker Compose
- **Monitoring**: Prometheus, Grafana, Datadog, New Relic
- **Tracing**: Jaeger, Zipkin, OpenTelemetry

### Books
- "Continuous Delivery" by Jez Humble
- "The Phoenix Project" by Gene Kim
- "Building Microservices" by Sam Newman
- "Scrum: The Art of Doing Twice the Work in Half the Time"